https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/

In [0]:
# Uncomment when running locally (not in Google Colab)
# import sys
# !{sys.executable} -m pip install tensorflow
# !{sys.executable} -m pip install keras

In [1]:
import keras
from numpy import array
from pickle import dump

Using TensorFlow backend.


In [0]:
inputFile = "kvv.txt"
sequenceFile = "kvv_sequences.txt"

In [0]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [4]:
# load document
doc = load_doc(inputFile)
print(doc[:200])

 Berliner Vorlesungsverzeichnis Wissenschaftsgeschichte
Wintersemester 2019/20
www.wissensgeschichte-berlin.de

2

Liebe Studierende,
die vorliegende Zusammenstellung der aktuellen Lehrveranstaltungen


In [0]:
import string

# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [6]:
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['berliner', 'vorlesungsverzeichnis', 'wissenschaftsgeschichte', 'wintersemester', 'wwwwissensgeschichteberlinde', 'liebe', 'studierende', 'die', 'vorliegende', 'zusammenstellung', 'der', 'aktuellen', 'lehrveranstaltungen', 'zur', 'wissenschaftsgeschichte', 'richtet', 'sich', 'an', 'studierende', 'und', 'weitere', 'interessierte', 'sie', 'das', 'lehrangebot', 'der', 'drei', 'berliner', 'zusammen', 'und', 'zeigt', 'damit', 'die', 'reichhaltigkeit', 'und', 'der', 'wissenschaftshistorischen', 'themen', 'in', 'berlin', 'die', 'freie', 'die', 'und', 'die', 'technische', 'haben', 'im', 'juli', 'mit', 'der', 'maxplanckgesellschaft', 'vereinbart', 'das', 'zentrum', 'aufzubauen', 'das', 'vorliegende', 'kommentierte', 'vorlesungsverzeichnis', 'wissenschaftsgeschichte', 'diese', 'zusammenarbeit', 'der', 'drei', 'und', 'des', 'maxplanckinstituts', 'wissenschaftsgeschichte', 'in', 'diesem', 'feld', 'die', 'auswahl', 'der', 'lehrveranstaltungen', 'in', 'dieser', 'fassung', 'basiert', 'auf', 'den', '

In [7]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 26865


In [0]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [0]:
# save sequences to file
save_doc(sequences, sequenceFile)

In [0]:
doc = load_doc(sequenceFile)
lines = doc.split('\n')

In [0]:
# integer encode sequences of words
tokenizer = keras.preprocessing.text.Tokenizer(filters='\n')
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [0]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [0]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = keras.utils.to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [14]:
# define model
model = keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size, 50, input_length=seq_length))
model.add(keras.layers.LSTM(100, return_sequences=True))
model.add(keras.layers.LSTM(100))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(vocab_size, activation='softmax'))
print(model.summary())




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            318350    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 6367)              643067    
Total params: 1,112,317
Trainable params: 1,112,317
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Epoch 1/5
26865/26865 [==============================] - 39s 1ms/step - loss: 7.2535 - acc: 0.0398
Epoch 2/5
26865/26865 [==============================] - 37s 1ms/step - loss: 6.9731 - acc: 0.0410
Epoch 3/5
26865/26865 [==============================] - 37s 1ms/step - loss: 6.7391 - acc: 0.0449
Epoch 4/5
23296/26865 [=========================>....] - ETA: 5s - loss: 6.5733 - acc: 0.0524

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-97c08eb4ff88>", line 3, in <module>
    model.fit(X, y, batch_size=128, epochs=5)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 1178, in fit
    validation_freq=validation_freq)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training_arrays.py", line 204, in fit_loop
    outs = fit_function(ins_batch)
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 2979, in __call__
    return self._call(inputs)
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 2937, in _call
    fetched = self._callable_fn(*array_vals)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py", line 1472, in __call__
    run_metadata_ptr)
Keyboard

KeyboardInterrupt: ignored

In [17]:
# fit model
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
26865/26865 [==============================] - 38s 1ms/step - loss: 6.4428 - acc: 0.0583
Epoch 2/100
26865/26865 [==============================] - 37s 1ms/step - loss: 6.2860 - acc: 0.0661
Epoch 3/100
26865/26865 [==============================] - 37s 1ms/step - loss: 6.1363 - acc: 0.0752
Epoch 4/100
26865/26865 [==============================] - 37s 1ms/step - loss: 5.9782 - acc: 0.0834
Epoch 5/100
26865/26865 [==============================] - 38s 1ms/step - loss: 5.8297 - acc: 0.0896
Epoch 6/100
26865/26865 [==============================] - 38s 1ms/step - loss: 5.6977 - acc: 0.0948
Epoch 7/100
26865/26865 [==============================] - 39s 1ms/step - loss: 5.5700 - acc: 0.0983
Epoch 8/100
26865/26865 [==============================] - 40s 1ms/step - loss: 5.4523 - acc: 0.1028
Epoch 9/100
26865/26865 [==============================] - 39s 1ms/step - loss: 5.3343 - acc: 0.1083
Epoch 10/100
26865/26865 [==============================] - 39s 1ms/step - loss: 5.2362 - a

In [0]:
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))